In [1]:
# !pip install pandas numpy scikit-learn
!pip install tqdm


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

# Загрузка данных

В бейзлайне реализована простейшая модель на эго-графе.

Для каждой пары вершин u и v эго-графа попытаемся найти всех общих "друзей" w. Силой связи между вершинами u и v будем считать средную силу связи между ними и общими друзьями.

Поскольку обучение для такой модели не требуется, будем пользоваться только тестовой выборкой.

In [7]:
%%time

test = pd.read_csv("data/test.csv")
submission = pd.read_csv("data/submission.csv")

CPU times: total: 13.5 s
Wall time: 13.5 s


In [32]:
#test.head()
cur = submission[submission["ego_id"] == 8]
curt = test[test["ego_id"] == 8]

for u, v in zip(cur["u"], cur["v"]):
    
    res.append((u, v))

0 93
0 143
0 151
1 24
5 4
7 16
8 20
8 29
11 0
13 7
14 11
46 110
54 58
55 0
64 57
74 159
75 0
94 179
131 106
152 65


In [9]:
submission.head(100)

,ego_id,u,v,x1
0,8,0,93,0.000000
1,8,0,143,0.000000
2,8,0,151,1.606742
3,8,1,24,0.026496
4,8,5,4,0.159857
...,...,...,...,...
95,28,122,229,0.525159
96,28,123,110,0.009399
97,28,126,116,0.117795
98,28,130,92,0.070024


In [6]:
# get graph

for u, v in zip(submission_ego_net["u"], submission_ego_net["v"]):
    if 

SyntaxError: invalid syntax (3857053417.py, line 4)

Проверочные данные - недоступны участникам, используются для подсчёта метрики:

In [ ]:
control = pd.read_csv("data/control.csv")

Таблицы control и submission отличаются только последним столбцом x1. В таблице control в этом столбце содержатся истинные значения связей x1.

In [ ]:
control[["ego_id", "u", "v"]].equals(submission[["ego_id", "u", "v"]])

Таблица submission отсортирована по возрастанию ego_id, u, v:

In [ ]:
submission.equals(submission.sort_values(["ego_id", "u", "v"]))

In [ ]:
submission.head()

# Модель

In [ ]:
submission["ego_id"].value_counts()

Константное предсказание:

In [ ]:
%%time

submission_dummy = submission.copy()
submission_dummy["x1"] = np.nanmean(test["x1"].values)

In [ ]:
df = pd.DataFrame()
df['u'] = np.array([1, 2, 3, 4])
df['x1'] = np.array([10, 10, 10, 10])


df1 = pd.DataFrame()
df1['u'] = np.array([2, 3, 4])
df1['x1'] = np.array([19, 19, 19])


t = df1.merge(df, on='u')
np.mean(t.drop("u", axis=1).values, axis = None)




In [ ]:
%%time

from tqdm import tqdm


ego_id_list = submission["ego_id"].drop_duplicates().values
for ego_id in tqdm(ego_id_list):
    submission_ego_net = submission[submission["ego_id"] == ego_id]
    test_ego_net = test[test["ego_id"] == ego_id]
    friendship = np.zeros_like(submission_ego_net["x1"].values)
    for i, (u, v) in enumerate(zip(submission_ego_net["u"], submission_ego_net["v"])):
        u_x1 = test_ego_net.loc[test_ego_net["u"] == u, ["v", "x1"]].dropna()
        v_x1 = test_ego_net.loc[test_ego_net["u"] == v, ["v", "x1"]].dropna()
        common_friends = u_x1.merge(v_x1, on="v")
        if common_friends.shape[0] > 0:
            friendship[i] = np.mean(common_friends.drop("v", axis=1).values)
    submission.loc[submission["ego_id"] == ego_id, "x1"] = friendship

# Посчёт метрик

In [ ]:
def RMSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

In [ ]:
print("Dummy model RMSE: {}".format(RMSE(control["x1"], submission_dummy["x1"])))

In [ ]:
print("Baseline model RMSE: {}".format(RMSE(control["x1"], submission["x1"])))

In [ ]:
submission.head()

In [19]:
submission.to_csv('nmbits.csv')